In [3]:
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer, pipeline
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score


# Load your dataset
ds = load_dataset("higopires/RePro-categories-multilabel")

def filter_inadequada(example):
    return example["INADEQUADA"] == 0

ds = ds.filter(filter_inadequada)



In [2]:
classifier = pipeline("zero-shot-classification",mode="PORTULAN/albertina-1b5-portuguese-ptbr-encoder",    return_all_scores=True) #, model="facebook/bart-large-mnli")

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [4]:
def compute_metrics(predictions,labels):
    # Calculate sample-wise F1 score
    f1 = f1_score(labels, predictions, average='micro', zero_division=0)
    precision = precision_score(labels, predictions, average='micro', zero_division=0)

    recall = recall_score(labels, predictions, average='micro', zero_division=0)
    return {'f1_micro': float(f1), 'precision': float(precision), 'recall': float(recall)}
print(compute_metrics([[1,0,1],[0,1,0]],[[1,0,1],[0,1,0]]))

{'f1_micro': 1.0, 'precision': 1.0, 'recall': 1.0}


In [ ]:
preds = []
y = []
for i in range(len(ds['validation'])):
    if i % 100 == 0:
        print(f"Processing example {i}/{len(ds)}")
    
    example = ds['validation'][i]
    text = example["review_text"]
    
    # Perform zero-shot classification
    entrega = classifier(text, candidate_labels=["entrega", "não entrega"], multi_label=True)
    outros = classifier(text, candidate_labels=["outros", "não outros"], multi_label=True)
    produtos = classifier(text, candidate_labels=["produtos", "não produtos"], multi_label=True)
    condições_Recb = classifier(text, candidate_labels=["condições de recebimento", "não condições de recebimento"], multi_label=True)
    anuncio = classifier(text, candidate_labels=["anuncio", "não anuncio"], multi_label=True)
    labels = [entrega['labels'][0], outros['labels'][0], produtos['labels'][0], condições_Recb['labels'][0], anuncio['labels'][0]]
    # transform the labels to a binary format
    labels = [1 if label == "entrega" or label == "outros" or label == "produtos" or label == "condições de recebimento" or label == "anuncio" else 0 for label in labels]
    preds.append(labels)
    y.append([example["ENTREGA"],example["OUTROS"],example["PRODUTO"],example["CONDICOESDERECEBIMENTO"],example["ANUNCIO"]])
    #print(preds,y)
    #print(compute_metrics(preds,y))
print(compute_metrics(preds,y))



Processing example 0/3
[[0, 0, 0, 0, 0]] [[0, 0, 1, 0, 0]]
{'f1_micro': 0.0, 'precision': 0.0, 'recall': 0.0}
[[0, 0, 0, 0, 0], [1, 1, 1, 1, 1]] [[0, 0, 1, 0, 0], [1, 0, 1, 0, 0]]
{'f1_micro': 0.5, 'precision': 0.4, 'recall': 0.6666666666666666}
[[0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 1, 1]] [[0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 0]]
{'f1_micro': 0.46153846153846156, 'precision': 0.3333333333333333, 'recall': 0.75}
[[0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 1, 1], [0, 1, 1, 0, 1]] [[0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0]]
{'f1_micro': 0.3333333333333333, 'precision': 0.25, 'recall': 0.5}
[[0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 1, 1], [0, 1, 1, 0, 1], [0, 1, 1, 0, 1]] [[0, 0, 1, 0, 0], [1, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 1, 0, 0]]
{'f1_micro': 0.36363636363636365, 'precision': 0.26666666666666666, 'recall': 0.5714285714285714}
[[0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [1, 0, 1, 1, 1], [0, 1, 1, 0, 1], [0, 1, 1, 0, 1], [1, 1,